In [2]:
# System Libs
import multiprocessing as mp
import sys
import os
from glob import glob
from time import time
from pathlib import Path

# Other Libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.metrics import f1_score
from PIL import Image
from tqdm import tqdm

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import CenterCrop, Resize, ToTensor, Normalize
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Local Libs
from data_utils import MaskClassifierDataset
from models import BaseModel

cuda


In [3]:
# Path Setting
# Project Dir
dir_project = Path('/opt/ml')

# Code Dir
dir_code = dir_project.joinpath('code')

# Model Dir
dir_model = dir_project.joinpath('model')

# Code Data
dir_data = dir_project.joinpath('input/data')
dir_eval = dir_data.joinpath('eval')
dir_train = dir_data.joinpath('train')

# Train Data
dir_img = dir_train.joinpath('images')
file_train_raw = dir_train.joinpath('train.csv')
file_train = dir_train.joinpath('train_processed.csv')

In [4]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

<br><br><br>
# 1. Dataset & DataLoader
---

In [5]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

<br><br><br>
# 2. Model
---

In [6]:
model = BaseModel(num_classes=18)
model.load_state_dict(torch.load(dir_model.joinpath('model_base.pt')))
model.to(device)
model.eval()

BaseModel(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=64, out_features=32, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=32, out_features=18, bias=True)
  )
)

<br><br><br>
# 3. Inference
---

In [7]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose(
        [
            transforms.ToTensor(),
        ]
    )
dataset = TestDataset(image_paths, transform)
loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
# device = torch.device('cuda')
# model = MyModel(num_classes=18).to(device)
# model.eval()
# model = torch.load(dir_model.joinpath('model_01.pt'))
# print(model.eval())

In [8]:
# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

test inference is done!


In [9]:
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)

tensor([[[0.7569, 0.8275, 0.7373,  ..., 0.2157, 0.1961, 0.1725],
         [0.5020, 0.4471, 0.3608,  ..., 0.1961, 0.1843, 0.1686],
         [0.3412, 0.2549, 0.3176,  ..., 0.1765, 0.1765, 0.1725],
         ...,
         [0.0902, 0.0902, 0.0902,  ..., 0.1451, 0.1451, 0.1686],
         [0.0902, 0.0902, 0.0941,  ..., 0.1412, 0.1216, 0.1333],
         [0.1020, 0.1020, 0.1059,  ..., 0.1294, 0.0824, 0.0902]],

        [[0.7882, 0.8588, 0.7725,  ..., 0.2235, 0.2039, 0.1804],
         [0.5333, 0.4784, 0.3961,  ..., 0.2039, 0.1922, 0.1765],
         [0.3725, 0.2863, 0.3529,  ..., 0.1843, 0.1843, 0.1804],
         ...,
         [0.1059, 0.1059, 0.1059,  ..., 0.1608, 0.1608, 0.1843],
         [0.1059, 0.1059, 0.1098,  ..., 0.1569, 0.1373, 0.1490],
         [0.1176, 0.1176, 0.1216,  ..., 0.1451, 0.0980, 0.1059]],

        [[0.7961, 0.8667, 0.7686,  ..., 0.1804, 0.1608, 0.1373],
         [0.5412, 0.4863, 0.3922,  ..., 0.1608, 0.1490, 0.1333],
         [0.3804, 0.2941, 0.3490,  ..., 0.1412, 0.1412, 0.

In [16]:
l = iter(loader)
data = l.next()

In [19]:
with torch.no_grad():
    data = data.to(device)
    pred = model(images)

In [22]:
pred

tensor([[ 99584.9141, 100431.1641,  99776.9453,  99888.3281,  99624.9062,
         100044.6484,  99001.0469, 100536.5859,  99900.8516,  99356.3516,
          99655.2500,  99974.5625,  99492.0312, 100490.0234, 100170.6484,
         100375.2812,  99304.6797,  99252.2031]], device='cuda:0')